In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16

from keras_tqdm import TQDMNotebookCallback

from datetime import datetime
import os

import numpy as np
import pandas as pd
import math

pd.options.display.max_rows = 40

Using TensorFlow backend.


In [2]:
vgg16 = VGG16(weights = 'imagenet',include_top=False)
x = vgg16.get_layer('block5_conv3').output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model_final = Model(inputs=vgg16.input, outputs=x)
model_final.compile(optimizer=SGD(lr=0.00001, momentum=0.9, nesterov=True),  loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
model_final.load_weights('./weights-iter-3-epoch-09.hdf5')

In [4]:
for layer in model_final.layers:
   layer.trainable = True

In [5]:
# Image Data Generators
datagen = ImageDataGenerator( 
    featurewise_center            = True,
    rescale                       = 1.,
    rotation_range                = 10,
    width_shift_range             = .1,
    height_shift_range            = .1,
    shear_range                   = 0.2,
    zoom_range                    = 0.2,
    horizontal_flip               = True,
    vertical_flip                 = False,
    fill_mode                     = "reflect")

# normalization neccessary for correct image input to VGG16
datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

# no data augmentation for validation and test set
validgen = ImageDataGenerator(rescale=1., featurewise_center=True)
validgen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

In [6]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 600
img_height = 450

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_val   = 32

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

validation_samples = len(val_gen.filenames)

Found 568 images belonging to 2 classes.


In [7]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.081546935696729939, 0.98415492957746475]

In [8]:
batch_size_train = 6
batch_size_test  = 3

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)



test_gen = datagen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_test,
        class_mode  = "binary",
        shuffle     = True)

train_samples      = len(train_gen.filenames)

Found 1727 images belonging to 2 classes.
Found 1531 images belonging to 1 classes.


In [9]:
for i in range(math.ceil(train_samples/batch_size_train)):
    print('batch: ' + str(i))
    test_imgs, test_labels = next(test_gen)
    train_imgs, train_labels = next(train_gen)
    test_labels = model_final.predict(test_imgs)
    test_labels = np.array([1. if x>=0.5 else 0. for x in test_labels])
    full_batch_imgs = np.concatenate((train_imgs, test_imgs))
    full_batch_labels = np.concatenate((train_labels, test_labels))
    model_final.train_on_batch(full_batch_imgs, full_batch_labels)

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100

In [10]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.091945192684114524, 0.9859154929577465]

In [11]:
for i in range(math.ceil(train_samples/batch_size_train)):
    print('batch: ' + str(i))
    test_imgs, test_labels = next(test_gen)
    train_imgs, train_labels = next(train_gen)
    test_labels = model_final.predict(test_imgs)
    test_labels = np.array([1. if x>=0.5 else 0. for x in test_labels])
    full_batch_imgs = np.concatenate((train_imgs, test_imgs))
    full_batch_labels = np.concatenate((train_labels, test_labels))
    model_final.train_on_batch(full_batch_imgs, full_batch_labels)

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100

In [12]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.13198011181756697, 0.97359154929577463]

In [13]:
for i in range(math.ceil(train_samples/batch_size_train)):
    test_imgs, test_labels = next(test_gen)
    train_imgs, train_labels = next(train_gen)
    test_labels = model_final.predict(test_imgs)
    test_labels = np.array([1. if x>=0.5 else 0. for x in test_labels])
    full_batch_imgs = np.concatenate((train_imgs, test_imgs))
    full_batch_labels = np.concatenate((train_labels, test_labels))
    model_final.train_on_batch(full_batch_imgs, full_batch_labels)

In [14]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.091893477375519908, 0.98063380281690138]

In [15]:
for i in range(math.ceil(train_samples/batch_size_train)):
    test_imgs, test_labels = next(test_gen)
    train_imgs, train_labels = next(train_gen)
    test_labels = model_final.predict(test_imgs)
    test_labels = np.array([1. if x>=0.5 else 0. for x in test_labels])
    full_batch_imgs = np.concatenate((train_imgs, test_imgs))
    full_batch_labels = np.concatenate((train_labels, test_labels))
    model_final.train_on_batch(full_batch_imgs, full_batch_labels)

In [16]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.093184873635833915, 0.98239436619718312]

In [17]:
for i in range(math.ceil(train_samples/batch_size_train)):
    test_imgs, test_labels = next(test_gen)
    train_imgs, train_labels = next(train_gen)
    test_labels = model_final.predict(test_imgs)
    test_labels = np.array([1. if x>=0.5 else 0. for x in test_labels])
    full_batch_imgs = np.concatenate((train_imgs, test_imgs))
    full_batch_labels = np.concatenate((train_labels, test_labels))
    model_final.train_on_batch(full_batch_imgs, full_batch_labels)

In [18]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples/batch_size_val))

[0.10197546908297464, 0.98415492957746475]